In [1]:
#we will search gmail for specific emails and summarize the content using Azure OpenAIs model.

In [1]:
#pip install gmail and azure open ai clients
#install requirements.txt before running



In [1]:
import os
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build




In [2]:
#load the credentials file

#setr the scope to search email, compose emails, read emails



In [3]:
def get_gmail_service():
    creds = None
    credentials_file='client_secret_718290307376-1gn4toeqcsi4pstk24n6pedui6odo5cm.apps.googleusercontent.com.json'
    SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
    # 1) Try to load existing token.json (authorized user file)
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)

    # 2) If no valid creds, run browser OAuth flow using credentials.json
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            # Refresh existing token
            creds.refresh(Request())
        else:
            # First time: use the downloaded credentials.json
            flow = InstalledAppFlow.from_client_secrets_file(
                credentials_file, SCOPES
            )
            creds = flow.run_local_server(port=0)

        # Save the authorized user credentials for next time
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    # 3) Build the Gmail service
    service = build("gmail", "v1", credentials=creds)
    return service

In [4]:
service = get_gmail_service()

In [5]:
res = service.users().messages().list(
    userId="me",
    maxResults=5
).execute()
print(res.get("messages", []))

[{'id': '19ad84ae1cf5c2ac', 'threadId': '19ad84ae1cf5c2ac'}, {'id': '19ad84789651f24b', 'threadId': '19ad84789651f24b'}, {'id': '19ad82fdf9ce6e6c', 'threadId': '19ad82fdf9ce6e6c'}, {'id': '19ad8164a6ddc630', 'threadId': '19ad8164a6ddc630'}, {'id': '19ad814d77a4152e', 'threadId': '19ad814d77a4152e'}]


In [6]:
#lets setup a pydantic model for the email content. The emails are about Primary Conmect Order updates from their suplliers and shippers. We want to extract the order number, supplier name, shipper name, estimated delivery date, and current status of the order.
from pydantic import BaseModel, Field
class OrderUpdateEmail(BaseModel):
    order_number: str = Field(..., description="The unique identifier for the order.")
    supplier_name: str = Field(..., description="The name of the supplier providing the order.")
    shipper_name: str = Field(..., description="The name of the shipping company handling the delivery.")
    estimated_delivery_date: str = Field(..., description="The expected date of delivery for the order.")
    current_status: str = Field(..., description="The current status of the order (e.g., shipped, in transit, delivered).")
    email_summary: str = Field(..., description="A brief summary of the email content.")
    email_received_date: str = Field(..., description="The date the email was received.")
    email_sender: str = Field(..., description="The sender of the email.")
    email_subject: str = Field(..., description="The subject line of the email.")
    email_body_snippet: str = Field(..., description="A snippet of the email body content.")
    full_email_body: str = Field(..., description="The full body content of the email.")





In [7]:
#lets create a mock email to search for in the gmail and then parse it into the pydantic model above.

mock_email_content = """
Subject: Order Update - Order #12345 from Supplier XYZ
From: supplierxyz@example.com
Date: Dec 1, 2025
To: Customer Support Primary Connect

Dear Primary Connect Team,

We are requesting an update on the status of Order #12345 placed on Nov 20, 2025. The order was scheduled for delivery by Dec 5, 2025, but we have not received any shipping confirmation yet. Please provide us with the current status of the order, including any tracking information if available.
Thank you for your prompt attention to this matter.
Best regards,
Supplier XYZ
"""

#lets create a follow up email becaues no reply was received.
follow_up_email_content = """
Subject: Follow-up on Order Update - Order #12345 from Supplier XYZ
From: supplierxyz@example.com
Date: Dec 8, 2025
To: Customer Support Primary Connect
Dear Primary Connect Team,
We are following up on our previous email regarding Order #12345 placed on Nov 20, 2025. As of today, we have not received any updates or shipping confirmation for this order, which was scheduled for delivery by Dec 5, 2025. We kindly request an urgent update on the status of the order and any available tracking information.
Thank you for your immediate attention to this matter.
Best regards,
Supplier XYZ
"""


In [8]:
#lets search gmail for anything related to order 12345
query = "Order #12345"
results = service.users().messages().list(userId="me", q=query, maxResults=10).execute()
messages = results.get("messages", [])
for message in messages:
    msg = service.users().messages().get(userId="me", id=message['id']).execute()
    print(msg['snippet'])


Dear Primary Connect Team, We are following up on our previous email regarding Order #12345 placed on Nov 20, 2025. As of today, we have not received any updates or shipping confirmation for this order
ear Primary Connect Team, We are requesting an update on the status of Order #12345 placed on Nov 20, 2025. The order was scheduled for delivery by Dec 5, 2025, but we have not received any shipping



In [9]:
#now lets send this to azure open ai to parse into the pydantic model
import asyncio
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

agent = AzureOpenAIChatClient(credential=AzureCliCredential()).create_agent(
    instructions="You are good at telling jokes.",
    name="Joker"
)

In [10]:
result = await agent.run("Tell me a joke about a pirate.")
print(result.text)

What's a pirate's favorite letter? You'd think it's "R," but their first love is the "C."


In [11]:
from typing import Annotated
from pydantic import Field
from agent_framework import ai_function

@ai_function(name="weather_tool", description="Retrieves weather information for any location")
def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    return f"The weather in {location} is cloudy with a high of 15°C."

In [12]:
#lets create the gmail search tool as an ai_function
@ai_function(name="gmail_search_tool", description="Searches Gmail for emails matching a query")
def search_gmail(
    query: Annotated[str, Field(description="The search query to find relevant emails in Gmail.")],
) -> str:
    results = service.users().messages().list(userId="me", q=query, maxResults=10).execute()
    messages = results.get("messages", [])
    email_snippets = []
    for message in messages:
        msg = service.users().messages().get(userId="me", id=message['id']).execute()
        email_snippets.append(msg['snippet'])
    return "\n".join(email_snippets)

In [13]:
#lets create an agent that uses the gmail search tool.
gmmail_agent = AzureOpenAIChatClient(credential=AzureCliCredential()).create_agent(
    instructions="You are an assistant that helps users find emails in their Gmail account.",
    name="gmail_agent",
    tools=[search_gmail]
)

In [15]:
#we want to find all emails related to order 12345 and summarize them using the pydantic model defined earlier.
result = await gmmail_agent.run("Find all emails related to order 12345 and summarize them.",response_format=OrderUpdateEmail)

In [16]:
#now lets print the result using the Pydantic model 
# class OrderUpdateEmail(BaseModel):
#     order_number: str = Field(..., description="The unique identifier for the order.")
#     supplier_name: str = Field(..., description="The name of the supplier providing the order.")
#     shipper_name: str = Field(..., description="The name of the shipping company handling the delivery.")
#     estimated_delivery_date: str = Field(..., description="The expected date of delivery for the order.")
#     current_status: str = Field(..., description="The current status of the order (e.g., shipped, in transit, delivered).")
#     email_summary: str = Field(..., description="A brief summary of the email content.")
#     email_received_date: str = Field(..., description="The date the email was received.")
#     email_sender: str = Field(..., description="The sender of the email.")
#     email_subject: str = Field(..., description="The subject line of the email.")
#     email_body_snippet: str = Field(..., description="A snippet of the email body content.")
#     full_email_body: str = Field(..., description="The full body content of the email.")

Order_info = OrderUpdateEmail.parse_raw(result.text)

print(f"""Order Number: {Order_info.order_number}
Supplier Name: {Order_info.supplier_name}
Shipper Name: {Order_info.shipper_name}
Estimated Delivery Date: {Order_info.estimated_delivery_date}
Current Status: {Order_info.current_status}
Email Summary: {Order_info.email_summary}
Email Received Date: {Order_info.email_received_date}
Email Sender: {Order_info.email_sender}
Email Subject: {Order_info.email_subject}
Email Body Snippet: {Order_info.email_body_snippet}
Full Email Body: {Order_info.full_email_body}
""")

Order Number: 12345
Supplier Name: Primary Connect (inferred)
Shipper Name: Unknown
Estimated Delivery Date: Dec 5, 2025
Current Status: Awaiting shipping confirmation — no update received
Email Summary: Two follow-up messages sent to Primary Connect requesting status updates for Order #12345 (placed Nov 20, 2025). The order was scheduled for delivery by Dec 5, 2025 but the sender reports no shipping confirmation or update has been received.
Email Received Date: Not provided
Email Sender: Unknown (customer contacting Primary Connect Team)
Email Subject: Not provided
Email Body Snippet: We are following up on our previous email regarding Order #12345 placed on Nov 20, 2025. As of today, we have not received any updates or shipping confirmation for this order.
Full Email Body: Email 1: "Dear Primary Connect Team, We are following up on our previous email regarding Order #12345 placed on Nov 20, 2025. As of today, we have not received any updates or shipping confirmation for this order"



C:\Users\vijoseph\AppData\Local\Temp\ipykernel_11736\1503338745.py:15: PydanticDeprecatedSince20: The `parse_raw` method is deprecated; if your data is JSON use `model_validate_json`, otherwise load the data then use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  Order_info = OrderUpdateEmail.parse_raw(result.text)


In [17]:
#Load the csv file mypc_data.csv
import pandas as pd
df = pd.read_csv('mypc_data.csv')
print(df.head())


   Order Number  Order Date Supplier Code          Supplier Name  \
0    4500123456  2025-01-15      SUP00123        Fresh Farms Ltd   
1    4500123498  2025-01-15      SUP00123        Fresh Farms Ltd   
2    4500123522  2025-01-16      VEG00999      GreenLeaf Produce   
3    4500123590  2025-01-16     MEAT00211       Country Meats Co   
4    4500123620  2025-01-17      SUP77121  Sunrise Foods Pty Ltd   

  Customer Site                        Site Name Pickup Date  \
0        DC0155        Woolworths Minchinbury DC  2025-01-16   
1        DC0542    Woolworths Melbourne Fresh DC  2025-01-16   
2        DC0155        Woolworths Minchinbury DC  2025-01-17   
3        DC0301  Woolworths Brisbane Regional DC  2025-01-17   
4        DC0633          Woolworths Adelaide RDC  2025-01-18   

  Delivery Window Start Delivery Window End Transport Mode  ...   Load ID  \
0      2025-01-17 06:00    2025-01-17 14:00        Ambient  ...  LD987654   
1      2025-01-17 03:00    2025-01-17 08:00   Refrig

In [18]:
#lets create a pydantic model for the mypc_data.csv

#    Order Number  Order Date Supplier Code          Supplier Name  \
# 0    4500123456  2025-01-15      SUP00123        Fresh Farms Ltd   
# 1    4500123498  2025-01-15      SUP00123        Fresh Farms Ltd   
# 2    4500123522  2025-01-16      VEG00999      GreenLeaf Produce   
# 3    4500123590  2025-01-16     MEAT00211       Country Meats Co   
# 4    4500123620  2025-01-17      SUP77121  Sunrise Foods Pty Ltd   

#   Customer Site                        Site Name Pickup Date  \
# 0        DC0155        Woolworths Minchinbury DC  2025-01-16   
# 1        DC0542    Woolworths Melbourne Fresh DC  2025-01-16   
# 2        DC0155        Woolworths Minchinbury DC  2025-01-17   
# 3        DC0301  Woolworths Brisbane Regional DC  2025-01-17   
# 4        DC0633          Woolworths Adelaide RDC  2025-01-18   

#   Delivery Window Start Delivery Window End Transport Mode  ...   Load ID  \
# 0      2025-01-17 06:00    2025-01-17 14:00        Ambient  ...  LD987654   
# 1      2025-01-17 03:00    2025-01-17 08:00   Refrigerated  ...  LD987655   
# 2      2025-01-18 06:00    2025-01-18 14:00   Refrigerated  ...  LD987656   
# 3      2025-01-18 02:00    2025-01-18 10:00   Refrigerated  ...  LD987657   
# 4      2025-01-19 04:00    2025-01-19 12:00        Ambient  ...  LD987658   

#   Shipment ID           Status  Status Timestamp Pallets Cartons  Weight (kg)  \
# 0    SH123456       In Transit  2025-01-16 11:45      12     480         3620   
# 1    SH123457        Delivered  2025-01-17 05:42       8     320         2400   
# 2    SH123458          Delayed  2025-01-17 18:20       6     240         1750   
# 3    SH123459  Awaiting Pickup  2025-01-16 20:10      14     560         4100   
# 4    SH123460           Booked  2025-01-17 22:48      10     400         2980   

#    Volume (m3)  Temperature Zone                 Reference Notes  
# 0         18.4           Ambient                  On-time pickup  
# 1         12.1           Chilled          Delivered early window  
# 2          9.0           Chilled  Delay due to vehicle breakdown  
# 3         20.2           Chilled       Carrier confirmed booking  
# 4         13.8           Ambient    Awaiting pickup confirmation  

#lets create a pydantic model for this data

class OrderData(BaseModel):
    order_number: str = Field(..., description="The unique identifier for the order.")
    order_date: str = Field(..., description="The date the order was placed.")
    supplier_code: str = Field(..., description="The code of the supplier.")
    supplier_name: str = Field(..., description="The name of the supplier.")
    customer_site: str = Field(..., description="The customer site code.")
    site_name: str = Field(..., description="The name of the customer site.")
    pickup_date: str = Field(..., description="The date the order is scheduled for pickup.")
    delivery_window_start: str = Field(..., description="The start time of the delivery window.")
    delivery_window_end: str = Field(..., description="The end time of the delivery window.")
    transport_mode: str = Field(..., description="The mode of transport for the delivery.")
    load_id: str = Field(..., description="The load identifier.")
    shipment_id: str = Field(..., description="The shipment identifier.")
    status: str = Field(..., description="The current status of the shipment.")
    status_timestamp: str = Field(..., description="The timestamp of the current status.")
    pallets: int = Field(..., description="The number of pallets in the shipment.")
    cartons: int = Field(..., description="The number of cartons in the shipment.")
    weight_kg: float = Field(..., description="The weight of the shipment in kilograms.")
    volume_m3: float = Field(..., description="The volume of the shipment in cubic meters.")
    temperature_zone: str = Field(..., description="The temperature zone for the shipment.")
    reference_notes: str = Field(..., description="Additional reference notes for the shipment.")




In [19]:
#lets create a ai_tool that will search the csv file for order updates
@ai_function(name="csv_order_search_tool", description="Searches the mypc_data.csv file for order updates")
def search_csv_orders(
    order_number: Annotated[str, Field(description="The order number to search for in the CSV file.")],
) -> str:
    df_filtered = df[df['Order Number'] == int(order_number)]
    if df_filtered.empty:
        return "No order found with that order number."
    order = df_filtered.iloc[0]
    order_data = OrderData(
        order_number=str(order['Order Number']),
        order_date=order['Order Date'],
        supplier_code=order['Supplier Code'],
        supplier_name=order['Supplier Name'],
        customer_site=order['Customer Site'],
        site_name=order['Site Name'],
        pickup_date=order['Pickup Date'],
        delivery_window_start=order['Delivery Window Start'],
        delivery_window_end=order['Delivery Window End'],
        transport_mode=order['Transport Mode'],
        load_id=order['Load ID'],
        shipment_id=order['Shipment ID'],
        status=order['Status'],
        status_timestamp=order['Status Timestamp'],
        pallets=int(order['Pallets']),
        cartons=int(order['Cartons']),
        weight_kg=float(order['Weight (kg)']),
        volume_m3=float(order['Volume (m3)']),
        temperature_zone=order['Temperature Zone'],
        reference_notes=order['Reference Notes']
    )
    return order_data.json()




In [20]:
#lets create an agent that uses the csv order search tool
csv_agent = AzureOpenAIChatClient(credential=AzureCliCredential()).create_agent(
    
    instructions="You are an assistant that helps users find order updates from the mypc_data.csv file.",
    name="csv_order_agent",
    tools=[search_csv_orders]
)


In [21]:
#call the csv agent to find order 4500123456
result_csv = await csv_agent.run("Find the order update for order number 4500123456",response_format=OrderData)




C:\Users\vijoseph\AppData\Local\Temp\ipykernel_11736\2719656255.py:32: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  return order_data.json()


In [22]:
# the order data pydantic model is defined as:
# class OrderData(BaseModel):
#     order_number: str = Field(..., description="The unique identifier for the order.")
#     order_date: str = Field(..., description="The date the order was placed.")
#     supplier_code: str = Field(..., description="The code of the supplier.")
#     supplier_name: str = Field(..., description="The name of the supplier.")
#     customer_site: str = Field(..., description="The customer site code.")
#     site_name: str = Field(..., description="The name of the customer site.")
#     pickup_date: str = Field(..., description="The date the order is scheduled for pickup.")
#     delivery_window_start: str = Field(..., description="The start time of the delivery window.")
#     delivery_window_end: str = Field(..., description="The end time of the delivery window.")
#     transport_mode: str = Field(..., description="The mode of transport for the delivery.")
#     load_id: str = Field(..., description="The load identifier.")
#     shipment_id: str = Field(..., description="The shipment identifier.")
#     status: str = Field(..., description="The current status of the shipment.")
#     status_timestamp: str = Field(..., description="The timestamp of the current status.")
#     pallets: int = Field(..., description="The number of pallets in the shipment.")
#     cartons: int = Field(..., description="The number of cartons in the shipment.")
#     weight_kg: float = Field(..., description="The weight of the shipment in kilograms.")
#     volume_m3: float = Field(..., description="The volume of the shipment in cubic meters.")
#     temperature_zone: str = Field(..., description="The temperature zone for the shipment.")
#     reference_notes: str = Field(..., description="Additional reference notes for the shipment.")




result_csv_info = OrderData.parse_raw(result_csv.text)

print(f"""Order Number: {result_csv_info.order_number}
Order Date: {result_csv_info.order_date}
Supplier Code: {result_csv_info.supplier_code}
Supplier Name: {result_csv_info.supplier_name}
Customer Site: {result_csv_info.customer_site}
Site Name: {result_csv_info.site_name}
Pickup Date: {result_csv_info.pickup_date}
Delivery Window Start: {result_csv_info.delivery_window_start}
Delivery Window End: {result_csv_info.delivery_window_end}
Transport Mode: {result_csv_info.transport_mode}
Load ID: {result_csv_info.load_id}
Shipment ID: {result_csv_info.shipment_id}
Status: {result_csv_info.status}
Status Timestamp: {result_csv_info.status_timestamp}
Pallets: {result_csv_info.pallets}
Cartons: {result_csv_info.cartons}
Weight (kg): {result_csv_info.weight_kg}
Volume (m3): {result_csv_info.volume_m3}
Temperature Zone: {result_csv_info.temperature_zone}
Reference Notes: {result_csv_info.reference_notes}
""")







Order Number: 4500123456
Order Date: 2025-01-15
Supplier Code: SUP00123
Supplier Name: Fresh Farms Ltd
Customer Site: DC0155
Site Name: Woolworths Minchinbury DC
Pickup Date: 2025-01-16
Delivery Window Start: 2025-01-17 06:00
Delivery Window End: 2025-01-17 14:00
Transport Mode: Ambient
Load ID: LD987654
Shipment ID: SH123456
Status: In Transit
Status Timestamp: 2025-01-16 11:45
Pallets: 12
Cartons: 480
Weight (kg): 3620.0
Volume (m3): 18.4
Temperature Zone: Ambient
Reference Notes: On-time pickup



C:\Users\vijoseph\AppData\Local\Temp\ipykernel_11736\4226666953.py:27: PydanticDeprecatedSince20: The `parse_raw` method is deprecated; if your data is JSON use `model_validate_json`, otherwise load the data then use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  result_csv_info = OrderData.parse_raw(result_csv.text)


In [23]:

#https://learn.microsoft.com/en-us/python/api/agent-framework-core/agent_framework.magenticbuilder?view=agent-framework-python-latest

from agent_framework import MagenticBuilder, StandardMagenticManager


# # Create chat client for the manager
# manager_chat_client = AzureOpenAIChatClient(credential=AzureCliCredential())

# # Build Magentic workflow with agents
# workflow = (
#     MagenticBuilder()
#     .participants(gmail_search=gmmail_agent, csv_search=csv_agent)
#     .with_standard_manager(chat_client=manager_chat_client, max_round_count=20, max_stall_count=3)
#     .with_plan_review(enable=True)
#     .build()
# )


# async for update in workflow.run_stream("what is the status of order 12345?"):
#     if update.text:
#         print(update.text, end="", flush=True)

# # Execute workflow
# result_workflow = await workflow.run("what is the status of order 12345?")


# for message in result_workflow:
#     if hasattr(message, 'message') and hasattr(message.message, 'content'):
#         print(message.message.content)

In [24]:
# async for update in workflow.run_stream("what is the status of order 12345?"):
#     print(update.data, end="", flush=True)

In [25]:
# #this is the output of the workflow
# NoneNoneNone<agent_framework._types.ChatMessage object at 0x00000130B2F59550>_MagenticPlanReviewRequest(request_id='6e1e3b75-706d-4b49-8b7f-f8f053794217', task_text='what is the status of order 12345?', facts_text='1. GIVEN OR VERIFIED FACTS\n- The request references an order identified as "12345."\n- The user asks for the current status of that order.\n- No additional context was supplied (no company/store name, no customer identity, no platform, no dates, no tracking number).\n- I do not have direct access to any external order or shipping systems in this conversation.\n\n2. FACTS TO LOOK UP\n- The order record for order ID 12345 in the merchant’s order management system or e‑commerce platform (e.g., Shopify, Magento, internal OMS) — locate the order entry and its status field.\n- Payment transaction status in the payment gateway or processor (Stripe, PayPal, Adyen, etc.).\n- Fulfillment/warehouse status (picked, packed, shipped) from the fulfillment or WMS.\n- Shipping carrier and tracking number; real‑time tracking info on the carrier’s site (UPS, FedEx, USPS, DHL, etc.).\n- Order metadata: order date/time, shipping method/service level, items ordered, shipping address.\n- Any support/CRM tickets or notes tied to that order (Zendesk, Freshdesk, Salesforce) for cancellations, holds, or customer messages.\n- Fraud/risk flags or holds from fraud detection service.\n- Inventory records for the items in the order (availability or backorder status).\n- Carrier service alerts or local holiday/exception notices that could affect delivery times (carrier status pages).\n- Refund/return records if the order was cancelled or refunded.\n\n3. FACTS TO DERIVE\n- The user‑facing plain-language status (e.g., "Processing," "On hold," "Shipped — in transit," "Delivered," "Cancelled/Refunded") by mapping the OMS status code to business terminology.\n- Estimated delivery date by combining ship date, chosen service level, and carrier transit times (plus any carrier exceptions).\n- Whether fulfillment was blocked by payment failure, inventory shortage, or fraud hold by correlating payment, inventory, and fraud checks.\n- If a shipping label exists vs. actual carrier scan activity (label created without pickup vs. scans showing in‑transit or delivered).\n- Next required actions implied by the status (e.g., if "exception" on tracking, deduce likely causes such as address issue or attempted delivery failure).\n\n4. EDUCATED GUESSES\n- Typical possible statuses without lookup: "Processing" (order placed and paid, awaiting pick/pack), "Shipped" (label created and carrier scans show transit), "Delivered" (carrier delivered), "On hold" (payment/verification/inventory/fraud issue), or "Cancelled/Refunded."\n- If payment succeeded and the order was placed recently, the most common immediate states are "Processing" or "Shipped."\n- If a shipping label has been created but no carrier scans appear, common causes are label created but not yet picked up by carrier.\n- If the order number format is short/simple (e.g., 12345), it could be either an older order or a low-volume/test order, but that is speculative and not reliable.\n- Frequent causes of delay: inventory backorder, carrier transit delays, address problems, or fraud/verification holds.', plan_text='- csv_search: Search all available order/export CSVs for exact matches of "12345" (and common variants like "Order 12345", "#12345") — pull order row(s) and extract status, order date, payment status, shipping method, tracking number, and any notes.\n- gmail_search: Search the user\'s Gmail for messages containing "12345", "Order #12345", and merchant names (if known); prioritize order confirmations, shipping notifications, delivery receipts, and support threads — extract timestamps, status phrases, carrier/tracking numbers, and any cancellation/refund messages.\n- Reconcile results: if CSV and email both have info, take the most recent status by timestamp and report a concise user-facing status (e.g., Processing, Shipped — in transit, Delivered, On hold, Cancelled/Refunded) plus evidence (source, date, tracking).\n- If a tracking number is found, provide the carrier name and tracking number and recommend/check carrier tracking (or give the user the tracking link to follow).\n- If neither agent finds the order or results are ambiguous, request missing details from the user (merchant/store name, account email used to order, date range, or attach the order confirmation) and note any required permissions to access their data.', round_index=0)NoneNoneNone

#lets use this information to improve the workflow output for the Plan and Execution steps.

# async for update in workflow.run_stream("what is the status of order 12345?"):
#     print(update.data, end="", flush=True)


In [29]:
from typing import Optional

from agent_framework import (
    MagenticAgentDeltaEvent,
    MagenticAgentMessageEvent,
    MagenticOrchestratorMessageEvent,
    MagenticFinalResultEvent,
    RequestInfoEvent,
    MagenticPlanReviewRequest,
)
from agent_framework import MagenticBuilder
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

task = "what is the status of order 12345? Check both email updates and the order CSV data."

# Manager chat client
manager_chat_client = AzureOpenAIChatClient(credential=AzureCliCredential())

# Build workflow (plan review OFF for now to reduce noise)
workflow = (
    MagenticBuilder()
    .participants(gmail_search=gmmail_agent, csv_search=csv_agent)
    .with_standard_manager(
        chat_client=manager_chat_client,
        max_round_count=20,
        max_stall_count=3,
    )
    # Turn this back on later if you want to handle plan review explicitly
    # .with_plan_review(enable=True)
    .build()
)

async def run_workflow():
    last_stream_agent_id: Optional[str] = None
    stream_line_open: bool = False

    async for event in workflow.run_stream(task):

        # Orchestrator messages (planning / routing info)
        if isinstance(event, MagenticOrchestratorMessageEvent):
            text = (event.message.text or "").strip()
            if text:
                print(f"\n[ORCH:{event.kind}]\n{text}\n" + "-" * 40)

        # Streaming tokens from an agent
        elif isinstance(event, MagenticAgentDeltaEvent):
            if last_stream_agent_id != event.agent_id or not stream_line_open:
                if stream_line_open:
                    print()
                print(f"\n[STREAM:{event.agent_id}]: ", end="", flush=True)
                last_stream_agent_id = event.agent_id
                stream_line_open = True
            print(event.text, end="", flush=True)

        # Full agent message (end of an agent turn)
        elif isinstance(event, MagenticAgentMessageEvent):
            if stream_line_open:
                print(" (final)")
                stream_line_open = False
            msg = event.message
            if msg and msg.text:
                text = msg.text.replace("\n", " ")
                print(f"\n[AGENT:{event.agent_id}] {msg.role.value}\n{text}\n" + "-" * 40)

        # Manager’s synthesized final answer
        elif isinstance(event, MagenticFinalResultEvent):
            print("\n=== MANAGER FINAL RESULT ===\n")
            print(event.message.text)

        # If you re-enable plan review later, auto-accept it like this:
        elif isinstance(event, RequestInfoEvent) and isinstance(event.request, MagenticPlanReviewRequest):
            await event.reply(accept=True)

# In a notebook cell:
await run_workflow()



[ORCH:user_task]
what is the status of order 12345? Check both email updates and the order CSV data.
----------------------------------------

[ORCH:task_ledger]
We are working to address the following user request:

what is the status of order 12345? Check both email updates and the order CSV data.


To answer this request we have assembled the following team:

- gmail_search: Agent gmail_search
- csv_search: Agent csv_search


Here is an initial fact sheet to consider:

1. GIVEN OR VERIFIED FACTS
- The order identifier in the request is "12345".
- The user asks to check both email updates and the order CSV data for the status of that order.

2. FACTS TO LOOK UP
- Email messages mentioning order 12345 in the user's inbox (where to look: Gmail/Outlook/other provider; search terms: "12345", "order 12345", the merchant name, "order confirmation", "shipping confirmation", "delivery").
- Specific senders to check (where to look: mail folders and spam): orders@<merchant>, noreply@<merchant

C:\Users\vijoseph\AppData\Local\Temp\ipykernel_11736\2719656255.py:32: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  return order_data.json()


I searched the available order CSV dataset and found a match for order_id / order number 12345.

1) Full CSV row (fields and values returned)
- order_number: 12345
- order_date: 2025-01-20
- supplier_code: SUP55555
- supplier_name: Allied Foods Pty Ltd
- customer_site: DC0155
- site_name: Woolworths Minchinbury DC
- pickup_date: 2025-01-20
- delivery_window_start: 2025-01-21 06:00
- delivery_window_end: 2025-01-21 14:00
- transport_mode: Ambient
- load_id: LD990001
- shipment_id: SH990001
- status: Booked
- status_timestamp: 2025-01-20 09:12
- pallets: 7
- cartons: 280
- weight_kg: 2050.0
- volume_m3: 10.2
- temperature_zone: Ambient
- reference_notes: New order added for testing

2) Values for the specific fields you requested (present / not present)
- status: Booked
- fulfillment_status: not present
- order_status: not present
- tracking_number: not present
- carrier: not present
- shipping_date: not present (closest related fields: pickup_date = 2025-01-20, order_date = 2025-01-20)


GeneratorExit: 

GeneratorExit: 

 conversions for the timestamps present)
- order_date: 2025-01-20
- pickup_date: 2025-01-20
- delivery_window_start: 2025-01-21T06:00:00
- delivery_window_end: 2025-01-21T14:00:00
- status_timestamp: 2025-01-20T09:12:00

3) CSV file name and file modification timestamp
- File name / modification timestamp: not available from the CSV search tool output I used. The search returned the row contents but did not include the source file path or file modification time.

4) Multiple matches / authoritative/latest
- Only one matching row was returned, so it is the authoritative match in this dataset.
- If multiple files/rows had been returned I would compare the last_updated / status_timestamp fields and (where available) file modification timestamps to pick the latest; in this case status_timestamp = 2025-01-20T09:12:00 is the latest timestamp available in the row.

Files / paths searched
- The CSV search used the dataset index available to this environment (no per-file metadata was returned).

[2025-12-01 16:59:33 - c:\Users\vijoseph\AppData\Local\miniconda3\envs\gmail-agents\Lib\site-packages\agent_framework\_workflows\_magentic.py:1578 - ERROR] Magentic Orchestrator: Max round count reached


 (final)

[AGENT:gmail_search] assistant
I’m ready to run the exhaustive mailbox extraction as soon as you provide two things. Please pick one Gmail option (A or B) and also provide the CSV file/path and permissions requested below.  1) Choose one Gmail option (pick A or B) A — Programmatic read-only access (preferred) - Grant an app the Gmail readonly OAuth scope: https://www.googleapis.com/auth/gmail.readonly - Confirm exactly which account you will authorize (email address). - Confirm the recipient email address(es)/aliases to include for DSN/bounce searches (the address(es) that would appear as the envelope recipient; e.g., you@domain.com and any aliases). - Note: If you want, I can provide step‑by‑step instructions to create/authorize the app — say “help me authorize” and I’ll send that.  OR  B — Manual message copies (private, no mailbox access) - Forward candidate messages as attachments (Gmail: open message → three-dot menu → Forward as attachment) and upload/send those attachm